In [1]:
# Remove existing installations
!pip uninstall -y torch torchvision opacus psutil

# Install old versions with specific CUDA build
!pip install --quiet torch torchvision opacus
#!pip install torchcsprng
!pip install --quiet psutil

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: psutil 5.9.5
Uninstalling psutil-5.9.5:
  Successfully uninstalled psutil-5.9.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.8 M

In [ ]:
# Restart the runtime to ensure new installations are loaded correctly
import os
os.kill(os.getpid(), 9)

In [1]:
# After restarting, run this cell:
import torch
import opacus
from opacus import PrivacyEngine

In [2]:
# @title Default title text
# Verify versions and CUDA
print(f"PyTorch version: {torch.__version__}")
print(f"Opacus version: {opacus.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.5.1+cu124
Opacus version: 1.5.2
CUDA available: True


In [3]:
# Show GPU info
!nvidia-smi

Fri Nov 29 01:25:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              44W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
# Test CUDA setup
device = torch.device("cuda")
print(f"\nUsing device: {device}")
print(f"Device properties:")
print(f"  Name: {torch.cuda.get_device_name(0)}")
print(f"  Memory allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
print(f"  Memory cached: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")


Using device: cuda
Device properties:
  Name: NVIDIA A100-SXM4-40GB
  Memory allocated: 0.00 MB
  Memory cached: 0.00 MB


In [ ]:
# Try a minimal DP setup
model = torch.nn.Linear(10, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
privacy_engine = PrivacyEngine()

model.to(device)
print("\nBasic DP setup successful")


Basic DP setup successful


/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


In [5]:
from google.colab import files

# Upload the dpsgd_rng_benchmark.py and model.py
uploaded = files.upload()

Saving dpsgd_rng_benchmark.py to dpsgd_rng_benchmark.py
Saving model.py to model.py


In [6]:
!ls *.py

dpsgd_rng_benchmark.py	model.py


In [7]:
from dpsgd_rng_benchmark import main

main("resnet")
main("resnet", poisson_sampling=False)


Running resnet benchmark WITH poisson subsampling
Initializing privacy engine...
Creating dummy dataset...
Setting up private model and optimizer...
Private model setup complete.

Warmup...

DPSGD Settings:
Privacy Engine noise_multiplier: 1.0
Optimizer noise_multiplier: 1.0


/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)



Noise Generation std parameter: 1.0

DPSGD_NO_NOISE Settings:
Privacy Engine noise_multiplier: 0.0
Optimizer noise_multiplier: 0.0

Running main benchmark...
Iteration 0/100
Iteration 10/100
Iteration 20/100
Iteration 30/100
Iteration 40/100
Iteration 50/100
Iteration 60/100
Iteration 70/100
Iteration 80/100
Iteration 90/100

SAMPLING MODE: POISSON


Benchmark Results:
--------------------------------------------------------------------------------

DPSGD:
Average step time: 40.13 ms (± 6.60 ms)
Throughput: 797.39 examples/sec

Memory Usage:
CPU Memory: 1132.79 MB

Peak Memory Usage:
Peak CPU Memory: 0.00 MB

DPSGD_NO_NOISE:
Average step time: 40.30 ms (± 9.17 ms)
Throughput: 794.01 examples/sec

Memory Usage:
CPU Memory: 1132.85 MB

Peak Memory Usage:
Peak CPU Memory: 0.00 MB

Detailed Timing Statistics:
--------------------------------------------------------------------------------
data_sampling       :     1.14 ±   0.15
forward_backward    :    31.21 ±   7.28
optimizer_step      :

In [8]:
main("transformer")
main("transformer", poisson_sampling=False)


Running transformer benchmark WITH poisson subsampling
Initializing privacy engine...
Creating dummy dataset...
Setting up private model and optimizer...
Private model setup complete.

Warmup...

DPSGD Settings:
Privacy Engine noise_multiplier: 1.0
Optimizer noise_multiplier: 1.0

DPSGD_NO_NOISE Settings:
Privacy Engine noise_multiplier: 0.0
Optimizer noise_multiplier: 0.0

Noise Generation std parameter: 0.0

Running main benchmark...
Iteration 0/100
Iteration 10/100
Iteration 20/100
Iteration 30/100
Iteration 40/100
Iteration 50/100
Iteration 60/100
Iteration 70/100
Iteration 80/100
Iteration 90/100

SAMPLING MODE: POISSON


Benchmark Results:
--------------------------------------------------------------------------------

DPSGD:
Average step time: 18.66 ms (± 1.73 ms)
Throughput: 1715.20 examples/sec

Memory Usage:
CPU Memory: 1150.00 MB

Peak Memory Usage:
Peak CPU Memory: 0.00 MB

DPSGD_NO_NOISE:
Average step time: 18.49 ms (± 1.46 ms)
Throughput: 1730.93 examples/sec

Memory Us